## Week 5, Lab 2: Tackling an NLP Problem with Naive Bayes
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we are going to apply a **new** modeling technique to natural language processing data.

> "But how can we apply a modeling technique we haven't learned?!"

The DSI program is great - but we can't teach you *everything* about data science in 12 weeks! This lab is designed to help you start learning something new without it being taught in a formal lesson. 
- Later in the cohort (like for your capstone!), you'll be exploring models, libraries, and resources that you haven't been explicitly taught.
- After the program, you'll want to continue developing your skills. Being comfortable with documentation and being confident in your ability to read something new and decide whether or not it is an appropriate method for the problem you're trying to solve is **incredibly** valuable.

## Step 1: Define the problem.

Many organizations have a substantial interest in classifying users of their product into groups. Some examples:
- A company that serves as a marketplace may want to predict who is likely to purchase a certain type of product on their platform, like books, cars, or food.
- An application developer may want to identify which individuals are willing to pay money for "bonus features" or to upgrade their app.
- A social media organization may want to identify who generates the highest rate of content that later goes "viral."

### Summary
In this lab, you're an engineer for Facebook. In recent years, the organization Cambridge Analytica gained worldwide notoriety for its use of Facebook data in an attempt to sway electoral outcomes.

Cambridge Analytica, an organization staffed with lots of Ph.D. researchers, used the Big5 personality groupings (also called OCEAN) to group people into one of 32 different groups.
- The five qualities measured by this personality assessment are:
    - **O**penness
    - **C**onscientiousness
    - **E**xtroversion
    - **A**greeableness
    - **N**euroticism
- Each person could be classified as "Yes" or "No" for each of the five qualities.
- This makes for 32 different potential combinations of qualities. ($2^5 = 32$)
- You don't have to check it out, but if you want to learn more about this personality assessment, head to [the Wikipedia page](https://en.wikipedia.org/wiki/Big_Five_personality_traits).
- There's also [a short (3-4 pages) academic paper describing part of this approach](./celli-al_wcpr13.pdf).

Cambridge Analytica's methodology was, roughly, the following:
- Gather a large amount of data from Facebook.
- Use this data to predict an individual's Big5 personality "grouping."
- Design political advertisements that would be particularly effective to that particular "grouping." (For example, are certain advertisements particularly effective toward people with specific personality traits?)

You want to know the **real-world problem**: "Is what Cambridge Analytica attempted to do actually possible, or is it junk science?"

However, we'll solve the related **data science problem**: "Are one's Facebook statuses predictive of whether or not one is agreeable?"
> Note: If Facebook statuses aren't predictive of one being agreeable (one of the OCEAN qualities), then Cambridge Analytica's approach won't work very well!

## Step 2: Obtain the data.

Obviously, there are plenty of opportunities to discuss the ethics surrounding this particular issue... so let's do that.

In [67]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words 
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB, MultinomialNB
import os
from sklearn.metrics import f1_score

In [2]:
pwd

'/Users/pwalesdi/Desktop/GA/Naive_Bayes_Lab'

In [3]:
data = pd.read_csv('./mypersonality_final.csv', encoding = 'ISO-8859-1')

In [4]:
data.head(9)

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
5,b7b7764cfa1c523e4e93ab2a79a946c4,www.thejokerblogs.com,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/16/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
6,b7b7764cfa1c523e4e93ab2a79a946c4,"saw a nun zombie, and liked it. Also, *PROPNAM...",2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/27/09 05:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
7,b7b7764cfa1c523e4e93ab2a79a946c4,is in Kentucky. 421 miles into her 1100 mile j...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/18/09 06:34 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
8,b7b7764cfa1c523e4e93ab2a79a946c4,was about to finish a digital painting before ...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/09/09 02:58 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


### 1. What is the difference between anonymity and confidentiality? All else held equal, which tends to keep people safer?

Being anonymous means that you are unknown. This is full protection and implies that the person being looked at, studied, examined, can not be identified. Being confidential means that you have an agreement with some party that they will keep your information from outside obeservers but that they do know you and who you are. Anonymity is definitly safer because your identity can, in no way, be determined.

### 2. Suppose that the "unique identifier" in the above data, the `#AUTHID`, is a randomly generated key so that it can never be connected back to the original poster. Have we guaranteed anonymity here? Why or why not?

I don't think we have quite enough information to know for sure. It seems like that unique identifier could be linked to an ip-address from a computer or to the actual status at a particular time. But If all the information that we have is contained in the DF above then it seems like it would be impossible to track back to the original user.

### 3. As an engineer for Facebook, you recognize that user data will be used by Facebook and by other organizations - that won't change. However, what are at least three recommendations you would bring to your manager to improve how data is used and shared? Be as specific as you can.

## Step 3: Explore the data.

- Note: For our $X$ variable, we will only use the `STATUS` variable. For our $Y$ variable, we will only use the `cAGR` variable.

### 4. Explore the data here.
> We aren't explicitly asking you to do specific EDA here, but what EDA would you generally do with this data? Do the EDA you usually would, especially if you know what the goal of this analysis is.

In [5]:
# Checking to see the baseline score for Agreeableness. Here we can see that 53% of respondents scored yes on agreeableness
data['cAGR'].value_counts(normalize=True)

y    0.531209
n    0.468791
Name: cAGR, dtype: float64

In [6]:
# Change the 'cAGR' variable into a  0 or 1. 
data['cAGR'] = data['cAGR'].map({'y':1, 'n':0})

In [7]:
type(data['STATUS'][2])

str

### 5. What is the difference between CountVectorizer and TFIDFVectorizer?

A TF-IDF score tells us which words are most discriminating between documents. Words that occur often in one document but don't occur in many documents contain a great deal of discriminating power. So it is a more sophisicated way to look at the frequency that words show up in the documents or the corpus, whereas count vectorizer is creating a vectorof counts of your data to check how many times that word or value showed up in the document or corpus.


### 6. What are stopwords?

Stop words are commonly used words that provide little or no legitimate information about the content of the text you are trying to analyze.

### 7. Give an example of when you might remove stopwords.

Anytime your text has lots of common words like 'the', 'and', 'if', etc. But also, when you have a very specific topic. Many you are trying to analyze if a post is about the Golden State Warriors or the Sacramento Kings, words like 'basketball', 'player', 'team' might not be relevant in discerning so you would want to remove them and focus on other words like 'Oakland', 'Bay', 'Curry' etc.

### 8. Give an example of when you might keep stopwords in your model.

For starters, if you have a small dataset, you might want to consider keeping stop words because they may contain more predictive power. When you are dealing with translation data they would be important because all of the words would matter and help with context creation.

## Step 4: Model the data.

We are going to fit two types of models: a logistic regression and [a Naive Bayes classifier](https://scikit-learn.org/stable/modules/naive_bayes.html).

**Reminder:** We will only use the feature `STATUS` to model `cAGR`.

### We want to attempt to fit our models on sixteen sets of features:

1. CountVectorizer with 100 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
2. CountVectorizer with 100 features, with English stopwords removed and with the default `ngram_range`.
3. CountVectorizer with 100 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
4. CountVectorizer with 100 features, with English stopwords kept in and with the default `ngram_range`.
5. CountVectorizer with 500 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
6. CountVectorizer with 500 features, with English stopwords removed and with the default `ngram_range`.
7. CountVectorizer with 500 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
8. CountVectorizer with 500 features, with English stopwords kept in and with the default `ngram_range`.
9. TFIDFVectorizer with 100 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
10. TFIDFVectorizer with 100 features, with English stopwords removed and with the default `ngram_range`.
11. TFIDFVectorizer with 100 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
12. TFIDFVectorizer with 100 features, with English stopwords kept in and with the default `ngram_range`.
13. TFIDFVectorizer with 500 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
14. TFIDFVectorizer with 500 features, with English stopwords removed and with the default `ngram_range`.
15. TFIDFVectorizer with 500 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
16. TFIDFVectorizer with 500 features, with English stopwords kept in and with the default `ngram_range`.

### 9. Rather than doing this directly (e.g. instantiating 16 different vectorizers), what have we learned about that might make this easier? Use it.

In [71]:
# We can use a pipeline and a Gridsearch to do this. I will do it below in 11.
X = data['STATUS']
y = data['cAGR']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=79)


In [49]:
X_test.shape

(2480,)

In [50]:
y_test.shape

(2480,)

### 10. What are some of the advantages of fitting a logistic regression model?

The logistic regression model has a regularization component in it so you can run it with Lasso and Ridge regularization.

### 11. Fit a logistic regression model and compare it to the baseline.

In [72]:
# We will use a pipeline to do this

pipe = Pipeline([
            ('cvec', CountVectorizer()),
            ('regression', LogisticRegression()),
])

pipe_params = {
    'cvec' : [CountVectorizer(), TfidfVectorizer()],
    'cvec__max_features': [100, 500],
    'cvec__stop_words': [None, 'english'],
    'cvec__ngram_range': [(1,2), (1,1)],
    'regression' : [LogisticRegression(penalty='l1', solver='liblinear'), LogisticRegression(penalty='l2', solver='liblinear'), MultinomialNB()],
}


gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3, verbose=1, n_jobs=2)

gs.fit(X_train, y_train)
print(f' Best Parameters: {gs.best_params_}')
print('')
print(f' Cross Validation Accuracy Score: {gs.best_score_}')
print(f' Training Data Accuracy Score: {gs.score(X_train, y_train)}')
print(f' Testing Data Accuracy Score: {gs.score(X_test, y_test)}')

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   11.8s
[Parallel(n_jobs=2)]: Done 144 out of 144 | elapsed:   29.0s finished


 Best Parameters: {'cvec': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=500, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None), 'cvec__max_features': 500, 'cvec__ngram_range': (1, 1), 'cvec__stop_words': None, 'regression': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)}

 Cross Validation Accuracy Score: 0.5464569046658599
 Training Data Accuracy Score: 0.6214871588005917
 Testing Data Accuracy Score: 0.5512096774193549


These models didn't do much better than the baseline model. The baseline model had an accuracy score of 0.53 and the Logistic Regression model had an accuracy score on the testing data of 0.55. So a slight improvement, but nothing significant

### Summary of Naive Bayes 

Naive Bayes is a classification technique that relies on probability to classify observations.
- It's based on a probability rule called **Bayes' Theorem**... thus, "**Bayes**."
- It makes an assumption that isn't often met, so it's "**naive**."

Despite being a model that relies on a naive assumption, it performs **really** well.
- [Interested in details? Read more here if you want.](https://www.cs.unb.ca/~hzhang/publications/FLAIRS04ZhangH.pdf)


The [sklearn documentation](https://scikit-learn.org/stable/modules/naive_bayes.html) is here, but it can be intimidating. So, to quickly summarize the Bayes and Naive parts of the model...

#### Bayes' Theorem
If you've seen Bayes' Theorem, it relates the conditional probability of $P(A|B)$ to $P(B|A)$. (Don't worry; we won't be doing any probability calculations by hand! However, you may want to refresh your memory on conditional probability from our earlier lessons if you forget what a conditional probability is.)

$$
\begin{eqnarray*}
\text{Bayes' Theorem: } P(A|B) &=& \frac{P(B|A)P(A)}{P(B)}
\end{eqnarray*}
$$

- Let $A$ be that someone is "agreeable," like the OCEAN category.
- Let $B$ represent the words used in their Facebook post.

$$
\begin{eqnarray*}
\text{Bayes' Theorem: } P(A|B) &=& \frac{P(B|A)P(A)}{P(B)} \\
\Rightarrow P(\text{person is agreeable}|\text{words in Facebook post}) &=& \frac{P(\text{words in Facebook post}|\text{person is agreeable})P(\text{person is agreeable})}{P(\text{words in Facebook post})}
\end{eqnarray*}
$$

We want to calculate the probability that someone is agreeable **given** the words that they used in their Facebook post! (Rather than calculating this probability by hand, this is done under the hood and we can just see the results by checking `.predict_proba()`.) However, this is exactly what our model is doing. We can (a.k.a. the model can) calculate the pieces on the right-hand side of the equation to give us a probability estimate of how likely someone is to be agreeable given their Facebook post.

#### Naive Assumption

If our goal is to estimate $P(\text{person is agreeable}|\text{words in Facebook post})$, that can be quite tricky.

---

<details><summary>Bonus: if you want to understand why that's complicated, click here.</summary>
    
- The event $\text{"words in Facebook post"}$ is a complicated event to calculate.

- If a Facebook post has 100 words in it, then the event $\text{"words in Facebook post"} = \text{"word 1 is in the Facebook post" and "word 2 is in the Facebook post" and }\ldots \text{ and "word 100 is in the Facebook post"}$.

- To calculate the joint probability of all 100 words being in the Facebook post gets complicated pretty quickly. (Refer back to the probability notes on how to calculate the joint probability of two events if you want to see more.)
</details>

---

To simplify matters, we make an assumption: **we assume that all of our features are independent of one another.**

In some contexts, this assumption might be realistic!

### 12. Why would this assumption not be realistic with NLP data?

Becuase sentence structure is pretty dependent. Some words often proceed others and some words can't follow other words. So if we assume that Facebook posts are generally in sentence form, than this may be a faulty assumption.

Despite this assumption not being realistic with NLP data, we still use Naive Bayes pretty frequently.
- It's a very fast modeling algorithm. (which is great especially when we have lots of features and/or lots of data!)
- It is often an excellent classifier, outperforming more complicated models.

There are three common types of Naive Bayes models: Bernoulli Naive Bayes, Multinomial Naive Bayes, and Gaussian Naive Bayes.
- How do we pick which of the three models to use? It depends on our $X$ variable.
    - Bernoulli Naive Bayes is appropriate when our features are all 0/1 variables.
        - [Bernoulli NB Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB)
    - Multinomial Naive Bayes is appropriate when our features are variables that take on only positive integer counts.
        - [Multinomial NB Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB)
    - Gaussian Naive Bayes is appropriate when our features are Normally distributed variables. (Realistically, though, we kind of use Gaussian whenever neither Bernoulli nor Multinomial works.)
        - [Gaussian NB Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)

We are going to use Multinomial NB becayse all of our features are going to take on positive integer counts because we are creating an NLP model. We are going to be running the model on a sparse martix so this should work well. 

### 13. Suppose you CountVectorized your features. Which Naive Bayes model would be most appropriate to fit? Why? Fit it.

We are going to use Multinomial NB becayse all of our features are going to take on positive integer counts because we are creating an NLP model. We are going to be running the model on a sparse martix so this should work well. 

### 14. Suppose you TFIDFVectorized your features. Which Naive Bayes model would be most appropriate to fit? Why? Fit it.

I can use both the Multinomial and Gaussian NB. The Tfifd takes on values that are between 0 and 1 and represent the weight of the particular feature within the classificaion model

### 15. Compare the performance of your models.

In [57]:
# Instantiating a TfidfVectorizer to use with the Gaussian NB model. 
tvec = TfidfVectorizer()
tvec.fit(X_train, y_train)
X_train = tvec.transform(X_train)
X_train_df = pd.DataFrame(X_train.toarray(), # I have to be sure to convert the sparse matriz to an array so that it can be used by the GNB model
                                 columns = tvec.get_feature_names())
# Making sure that I ransform my test data set as well
X_test = tvec.transform(X_test) 
X_test_df = pd.DataFrame(X_test.toarray(),
                                 columns = tvec.get_feature_names())


In [66]:
X_test_df.describe()

,00,000,000ft,00am,00pm,01,02,02pm,03,0330,...,ça,échangé,équipe,étais,était,étant,état,être,østersund,últimas
count,2480.000000,2480.000000,2480.0,2480.0,2480.000000,2480.0,2480.0,2480.0,2480.0,2480.0,...,2480.000000,2480.0,2480.000000,2480.0,2480.0,2480.0,2480.0,2480.0,2480.0,2480.0
mean,0.000364,0.000513,0.0,0.0,0.000319,0.0,0.0,0.0,0.0,0.0,...,0.000272,0.0,0.000096,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.009103,0.011766,0.0,0.0,0.011984,0.0,0.0,0.0,0.0,0.0,...,0.007871,0.0,0.004765,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.261062,0.347273,0.0,0.0,0.541913,0.0,0.0,0.0,0.0,0.0,...,0.247289,0.0,0.237273,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
# Instantiating the GNB
gnb = GaussianNB()
gnb.fit(X_train_df, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [68]:
# Gett ing the predicted values for GNB
preds = gnb.predict(X_test_df)

In [63]:
# Scoring the testing vs. training data set
print(gnb.score(X_test_df, y_test))
print(gnb.score(X_train_df, y_train))


0.5536290322580645
0.8748151136210838


You can see the the model is really overfit. It is getting a 87% accuracy score on the training data but only a 55 percent accuracy score on the testing data. This is slightly better than the Logistic Regression and the Multinomial NB did in the pipeline, but there is no doubt that this model is going to have a much higher variance. I calculated an F1 score below to see how well it did....not very good at all. 

In [69]:
f1_score(y_test, preds)

0.4839160839160839

### 16. Even though we didn't explore the full extent of Cambridge Analytica's modeling, based on what we did here, how effective was their approach at using Facebook data to model agreeableness?

It doesn't seem to me like this approach was very effective. Perhaps they didn't care too much about hwo effective it was just that they could claim they did something but it seems pretty difficult to predict based on these methods we used. Perhaps if we were to have a much larger dataset and perhaps if we had multiple facebook status' from the same people we would be able to do this more effectively. 